In [11]:
from dataclasses import dataclass
import pulp
import pandas as pd
import yaml

@dataclass
class Armor:
    poise: int
    weight: float
    name: str
    location: str = None
    _var = None

    @property
    def var(self):
        if self._var is None:
            self._var = pulp.LpVariable(self.name, 0, 1, pulp.LpInteger)
        return self._var


def get_armor_from_pickle(armor_type):
    df = pd.read_pickle(f"{armor_type}.pkl")
    df["Poise"] = pd.to_numeric(df["Poise"], errors='coerce')
    df["Wgt."] = pd.to_numeric(df["Wgt."], errors='coerce')
    df.fillna(0, inplace=True)
    return [Armor(name = armor["Name"], poise=armor["Poise"], weight=armor["Wgt."], location=armor["How to Acquire"] ) for armor in df.to_dict(orient='records')]


def solve(weight_limit, my_armors_only = False):
    prob = pulp.LpProblem("all_armors", pulp.LpMaximize)

    armor_types = ["head", "chest", "arms", "legs"]
    if my_armors_only:
        with open('armor_values.yaml') as f:
            my_armors = yaml.load(f, Loader=yaml.Loader)
        typed_armor_lists = [[Armor(**armor) for armor in my_armors[armor_type]] for armor_type in armor_types]
    else:
        typed_armor_lists = [get_armor_from_pickle(armor_type) for armor_type in armor_types]

    # flatten list of lists into one big list
    armors = sum(typed_armor_lists, [])

    # limiting solution to max one of each armor type
    for armor_list in typed_armor_lists:
        prob += sum([a.var for a in armor_list]) <= 1.0

    # weight within limit
    weight = sum([armor.weight * armor.var for armor in armors])
    prob += weight <= weight_limit

    # maximise poise
    poise = sum([armor.poise * armor.var for armor in armors])
    prob += poise

    prob.solve(pulp.PULP_CBC_CMD(msg=0))


    print("Name - Poise - Weight ")

    for a in armors:
        if pulp.value(a.var) > 0:
            print(a.name, "-", a.poise, "-", a.weight)

    print("Total - ", pulp.value(poise), "-", pulp.value(weight))

    

In [12]:
solve(weight_limit=21.6, my_armors_only=True)

Name - Poise - Weight 
Knight Armor - 19 - 10.6
Beast Champion Greaves - 17 - 10.8
Total -  36.0 - 21.4
